<a href="https://colab.research.google.com/github/GustavoRizzo/Particle-Swarm-Optimization-PSO/blob/main/Particle_Swarm_Optimization_PSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Material inspirado em video do canal PyShine, https://www.youtube.com/watch?v=xEQv9YdvRiA&list=PLpCxkbETr8et3ouRXF2LlqobHMSb53_7b&index=25&ab_channel=PyShine


Problema: imagine um exame de animais, procurando uma fonte de comida. ninguem saber onde a fonte realemntes está. Então eles se espalham para procurar e o que tiver mais perto naquele momento lança um sinal para os demais, que vão tentar se aproximar desse sinal com o intuito de encontrar o alvo.

Pseudo código:
1) criar as particulas
2) avaliar a distancia do tergect de cada particula

Melhorias:poderia criar uma classe chamada ambiente/universo/word/mundo ela 

*   saberia tudo o que existe dentro dela: o tamanho do mundo; a posição de cada individua (caçador e caça)
*   a particula pode saber a quantidade de dimensões através da posição inicial (x0) passado para ela. E armazenar esse número dentro de uma variável interna chamada num_dimensions.


*   criar lógica para as cores, que de preferencia faça parecer um enxame 
*   melhorar a lógica de quando não oferecem uma velocidade incial a particula. Tentar usar map









# **Parte Lógica**

In [ ]:
# Bibliotecas
import random as rand
import numpy as np
from copy import deepcopy
import uuid 
from typing import List
import json
import math

In [ ]:
# Configurações
json_config = '''
{
  "max_iteracoes" : 50,
  "cacador" : {
    "max_vel" : 1.7,
    "coeficiente_inercia" : 0.7,
    "coeficiente_aceleracao_cognitivo" : 1,
    "coeficiente_aceleracao_social" : 2
  }
}
'''

config = json.loads(json_config)

In [ ]:
# Particula_2D
class Particula_2D:
  def __init__(self, pos_ini=[0.0, 0.0], vel_ini=[0.0, 0.0], ace_ini=[0.0, 0.0], nome=''):
    self.nome = nome
    self.pos = []
    self.vel = []
    self.ace = []
    self.pos.extend(pos_ini)    
    self.vel.extend(vel_ini)
    self.ace.extend(ace_ini)    

  def update_vel(self):
    self.vel = list(map(lambda v, a: v + a, self.vel, self.ace))

  def update_pos(self):
    self.pos = list(map(lambda p, v, a: ( p + v*1 + (1/2)*a*(1^2) ), self.pos, self.vel, self.ace))

  def update(self):
    self.update_pos()
    self.update_vel()
    

In [ ]:
class Cacador(Particula_2D):
  def __init__(self, pos_ini=[0.0, 0.0], vel_ini=[0.0, 0.0], nome=''):
    super().__init__(pos_ini, vel_ini, ace_ini=[0.0, 0.0], nome=nome)
    self.melhor_pos_pessoal = []
    self.melhor_pos_pessoal.extend(pos_ini)
    self.melhor_score = None


  def update(self, socre_atual, melhor_pos_social):
    self.update_pos()
    self.update_melhor_score(socre_atual)
    self.update_vel(melhor_pos_social)

    
  def update_vel(self, melhor_pos_social):
    w = config['cacador']['coeficiente_inercia'] 
    c1 = config['cacador']['coeficiente_aceleracao_cognitivo']
    c2 = config['cacador']['coeficiente_aceleracao_social']
    r1=rand.random()
    r2=rand.random()
    for i in range(0,2):
      cog_vel=r1*c1*(self.melhor_pos_pessoal[i]-self.pos[i])
      social_vel=r2*c2*(melhor_pos_social[i]-self.pos[i])
      self.vel[i]=w*self.vel[i]+cog_vel+social_vel
    self.limita_veleocidade_máxima()    
    return


  def limita_veleocidade_máxima(self):
    max_vel = config['cacador']['max_vel']
    v = np.array(self.vel)
    mod_vel = np.linalg.norm(v)
    if (mod_vel > max_vel):
      nova_vel = v * (max_vel/mod_vel) 
      self.vel = nova_vel.tolist()
      return
    else:
      return
  

  def update_melhor_score(self, score):
    if self.melhor_score is None or score > self.melhor_score :
      self.melhor_score = score
      self.melhor_pos_pessoal = self.pos

In [ ]:
def vertices_poligono(centro, num_lados, raio):
  rad = (2 * math.pi) / num_lados
  vertices = []
  for p in range(num_lados):
    px = centro[0] + raio * math.cos(rad *p)
    py = centro[1] + raio * math.sin(rad *p)
    vertices.append([px,py])
  return vertices

In [ ]:
# Iteracao
class Iteracao:
  def __init__(self, elementos: List[Particula_2D]):
    if ( all(isinstance(n, Particula_2D) for n in elementos) ):
      self.elementos= deepcopy(elementos)
    else:
      raise NameError('O argumneto passado deve ser uma lista de Particulas')

  
  @staticmethod
  def calcula_score(alvo, pos):
    a = np.array(alvo)
    b = np.array(pos)
    dist = np.linalg.norm(a-b)
    return  (-1)*dist


  @staticmethod
  def get_pos_alvo(elementos):
    lst_alvo = [e for e in elementos if e.nome == 'alvo']
    return lst_alvo[0].pos


  @staticmethod
  def get_pos_melhor_cacador(elementos):
    melhor_pos=[]
    maior_score = None
    pos_alvo = Iteracao.get_pos_alvo(elementos)
    lst_cacadores = [e for e in elementos if (isinstance(e, Cacador))]
    for c in lst_cacadores:
      rec = Iteracao.calcula_score(pos_alvo, c.pos)
      if (maior_score is None or rec > maior_score):
        maior_score = rec
        melhor_pos = c.pos
    return melhor_pos

In [ ]:
# Inicializa elementos da cena
elementos =[]

# Enxame
cordenadas = vertices_poligono([-3,-3], 12, 4)
for pos in cordenadas:
  c = Cacador(pos, nome=uuid.uuid4().hex)
  elementos.append(c)

# Alvo
alvo= Particula_2D( [8,8], nome='alvo')
elementos.append(alvo)

In [ ]:
# Processa o algoritimo de PSO
logIteracoes = []
logIteracoes.append( Iteracao(elementos) )

for i in range(config['max_iteracoes']):
  # atualiza todos que não são caçadores
  [e.update() for e in elementos if (not isinstance(e, Cacador))]  

  # atualiza caçadores
  alvo = Iteracao.get_pos_alvo(elementos)
  pos_mlehor_cacador = Iteracao.get_pos_melhor_cacador(elementos)
  lst_cacadores = [e for e in elementos if (isinstance(e, Cacador))]
  for c in lst_cacadores:
    score = Iteracao.calcula_score(alvo, c.pos)
    c.update(score, pos_mlehor_cacador)

  ite = Iteracao(elementos)
  logIteracoes.append( deepcopy(ite) )

In [ ]:
# Serialization logIteracoes
json_data = json.dumps(logIteracoes, default=lambda o: o.__dict__, indent=4)

# **Parte Visual**

In [ ]:
# Importa as bibliotecas resposnsaveis pela parte gráfica 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation, rc, lines
from IPython.display import HTML

In [ ]:
# Desserializar os dados vindos do PSO
iteracoes = json.loads(json_data)

# Determina a quantida de frames
num_frames = len(iteracoes)

In [ ]:
# Intancia cena com as suas dimensões
fig, ax = plt.subplots()

ax.set_xlim(( -10, 10))
ax.set_ylim((-10, 10))

In [ ]:
# Métodos auxiliares para criação das animação

def inicializaParticulas(iteracoes):
  elementos = iteracoes[0]['elementos']
  for id, pat in enumerate(elementos):
    p, = ax.plot([0], 'o')
    nome = elementos[id]['nome']
    p.set_label(nome)
  return tuple(ax.lines)


def atualizaPosicaoParticulas( frame , iteracoes):
  elementos_desenhados = ax.lines
  elementos = iteracoes[frame]['elementos']
  for id_pat, p in enumerate(elementos):
    label_teroica= elementos[id_pat]['nome']
    for e in elementos_desenhados:
      if( e.get_label() == label_teroica):
        e.set_data( elementos[id_pat]['pos'])
  return 

def animate(frame):
  atualizaPosicaoParticulas(frame , iteracoes)
  return tuple(ax.lines)

In [ ]:
# Inicializa a animação
anim = animation.FuncAnimation(fig, 
                               animate,
                               init_func = lambda: inicializaParticulas(iteracoes),
                               frames = num_frames, 
                               interval=200, 
                               blit=True)

In [ ]:
# Exibe a animação em HTML
HTML(anim.to_jshtml())